In [1]:
import sys
!{sys.executable} -m pip install --user torch
import torch
print(torch.__version__)
!{sys.executable} -m pip install --user torchvision
import torchvision
print(torchvision.__version__)


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
2.6.0+cu124

[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
0.21.0+cu124


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm  # For progress bars
import wandb
import json
import torchvision.models as models


################################################################################
# Part 3: Pretrained Model
################################################################################
class PretrainedResNet18(nn.Module):
    def __init__(self, num_classes=100):
        super(PretrainedResNet18, self).__init__()
        
        # Load ResNet18 with pretrained weights from ImageNet
        self.model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        
        # The first convolutional layer
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        
        # AI: Remove the default max pooling to preserve spatial resolution
        """The original ResNet18 applies a max pooling layer with a kernel size of 3 and a stride of 2 
        immediately after the first convolution, which significantly reduces the spatial resolution. 
        For 32×32 images, this pooling operation may remove too much spatial information. 
        Therefore, replacing it with `nn.Identity()` (i.e., no pooling) 
        helps preserve important details that are critical for classification."""
        self.model.maxpool = nn.Identity()
        
        # AI: Replace the simple fc layer with a sequence including dropout
        """Enhance the model's regularization and nonlinear representation capabilities 
        by adding intermediate fully connected layers, ReLU activations, Batch Normalization,
        and Dropout."""
        num_features = self.model.fc.in_features
        """Uses different regularization strategies (Dropout rate from high to low) 
        and batch normalization at different stages to ensure 
        that overfitting is strongly prevented in the initial stage
        while retaining more effective features in subsequent stages."""
        self.model.fc = nn.Sequential(
            nn.BatchNorm1d(num_features),  # Batch normalization for stability
            nn.Dropout(0.5),  # Increased dropout to reduce overfitting
            nn.Linear(num_features, 1024),  # Intermediate layer
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512),
            nn.Dropout(0.2),
            nn.Linear(512, num_classes)
        )


    def forward(self, x):
        return self.model(x)

################################################################################
# Define a one epoch training function
################################################################################
def train(epoch, model, trainloader, optimizer, criterion, CONFIG, scheduler=None):
    device = CONFIG["device"]
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct = 0
    total = 0
    
    # put the trainloader iterator in a tqdm so it can printprogress
    progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']} [Train]", leave=False)
    
    # iterate through all batches of one epoch
    for i, (inputs, labels) in enumerate(progress_bar):
        
        # move inputs and labels to the target device
        inputs, labels = inputs.to(device), labels.to(device)
        
        ### TODO - Your code here
        optimizer.zero_grad()
        # AI: Mixup augmentation (50% chance)
        if CONFIG.get("use_mixup", False) and np.random.random() < 0.5:
            # Generate mixup coefficient
            alpha = 0.2
            lam = np.random.beta(alpha, alpha)
            
            # Create mixed inputs and labels
            index = torch.randperm(inputs.size(0)).to(device)
            mixed_inputs = lam * inputs + (1 - lam) * inputs[index]
            
            # Forward pass with mixed inputs
            outputs = model(mixed_inputs)
            
            # Compute mixed loss
            loss = lam * criterion(outputs, labels) + (1 - lam) * criterion(outputs, labels[index])
        else:
            # Regular forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
       
        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        
       # Accumulate the loss value
        running_loss += loss.item()   ### TODO
        # Get the predicted classes
        _, predicted = outputs.max(1)    ### TODO
        
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        progress_bar.set_postfix({"loss": running_loss / (i + 1), "acc": 100. * correct / total})
        
    train_loss = running_loss / len(trainloader)
    train_acc = 100. * correct / total
    return train_loss, train_acc


################################################################################
# Define a validation function
################################################################################
def validate(model, valloader, criterion, device):
    """Validate the model"""
    model.eval() # Set to evaluation
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad(): # No need to track gradients
        
        # Put the valloader iterator in tqdm to print progress
        progress_bar = tqdm(valloader, desc="[Validate]", leave=False)

        # Iterate throught the validation set
        for i, (inputs, labels) in enumerate(progress_bar):
            
            # move inputs and labels to the target device
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs) ### TODO -- inference
            loss = criterion(outputs, labels)    ### TODO -- loss calculation

            running_loss += loss.item()  ### SOLUTION -- add loss from this sample
            _, predicted = torch.max(outputs, 1)   ### SOLUTION -- predict the class

            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

            progress_bar.set_postfix({"loss": running_loss / (i+1), "acc": 100. * correct / total})

    val_loss = running_loss/len(valloader)
    val_acc = 100. * correct / total
    return val_loss, val_acc

def main():
    ############################################################################
    # Configuration Dictionary
    # Adjust parameters for fine-tuning
    ############################################################################
    CONFIG = {
        "model": "PretrainedResNet18",  # Using the pretrained model
        "batch_size":128,
        "learning_rate": 5e-4,  # Lower learning rate for fine-tuning
        "weight_decay": 5e-4,
        "epochs": 50,  # Increase epochs for fine-tuning
        "num_workers": 4,
        "device": "mps" if torch.backends.mps.is_available()
        else "cuda" if torch.cuda.is_available() else "cpu",
        "data_dir": "./data",
        "ood_dir": "./data/ood-test",
        "wandb_project": "sp25-ds542-challenge",
        "use_mixup": True,
        "seed": 42,
    }
    # Set random seed for reproducibility
    torch.manual_seed(CONFIG["seed"])
    np.random.seed(CONFIG["seed"])
    
    import pprint
    print("\nCONFIG Dictionary:")
    pprint.pprint(CONFIG)

    ############################################################################
    # Data Transformation
    ############################################################################
    # Better model generalization
    # AI: Common improvement methods to improve the diversity of the training data and the generalization ability of the model
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.RandAugment(num_ops=2, magnitude=7),
        # AI: Add color/intensity augmentations
        transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
        transforms.RandomRotation(10), 
        
        transforms.ToTensor(),
        transforms.RandomErasing(p=0.05),  # Randomly erase parts of the image
        transforms.Normalize((0.5071, 0.4867, 0.4408),
                             (0.2675, 0.2565, 0.2761)),
    ])
    ###############
    # TODO Add validation and test transforms - NO augmentation for validation/test
    ###############
    
    # Validation and test transforms (NO augmentation)
    transform_test = transforms.Compose([
        transforms.ToTensor(),transforms.Normalize((0.5071, 0.4867, 0.4408),(0.2675, 0.2565, 0.2761))
    ]) ### TODO -- BEGIN SOLUTION

    ############################################################################
    # Data Loading
    ############################################################################
    
    trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                             download=True, transform=transform_train)
    
    # Split train into train and validation (80/20 split)
    train_size = int(0.8 * len(trainset))   ### TODO -- Calculate training set size
    val_size = len(trainset) - train_size     ### TODO -- Calculate validation set size
    trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])
    
    ### TODO -- define loaders and test set

    # Create Dataloader for training data
    trainloader = torch.utils.data.DataLoader(trainset, batch_size = CONFIG["batch_size"], shuffle = True, num_workers = CONFIG["num_workers"])
    # Create DataLoader for validation data
    valloader = torch.utils.data.DataLoader(valset, batch_size = CONFIG["batch_size"], shuffle = False, num_workers = CONFIG["num_workers"])

    # ... (Create validation and test loaders)
    testset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size = CONFIG["batch_size"], shuffle = False, num_workers = CONFIG["num_workers"])### TODO -- define loaders and test set

    ############################################################################
    # Instantiate model and move to target device
    ############################################################################
    model = PretrainedResNet18(num_classes=100)  ##TODO
    model = model.to(CONFIG["device"])
    
    print("\nModel summary:")
    print(f"{model}\n")
    
    # The following code you can run once to find the batch size that gives you the fastest throughput.
    # You only have to do this once for each machine you use, then you can just
    # set it in CONFIG.
    SEARCH_BATCH_SIZES = False
    if SEARCH_BATCH_SIZES:
        from utils import find_optimal_batch_size
        print("Finding optimal batch size...")
        optimal_batch_size = find_optimal_batch_size(model, trainset, CONFIG["device"], CONFIG["num_workers"])
        CONFIG["batch_size"] = optimal_batch_size
        print(f"Using batch size: {CONFIG['batch_size']}")

    ############################################################################
    # Loss Function, Optimizer and LR Scheduler
    ############################################################################
    # Use cross-entropy for classification
    # AI: For CIFAR-100, it's better to choose nn.CrossEntropyLoss
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)  ### TODO -- define loss criterion
    # Use AdamW optimizer
    # I compared two different optimizers, SGD and AdamW, and AdamW performed better in this case
    optimizer = optim.AdamW(model.parameters(), lr=CONFIG["learning_rate"], weight_decay=CONFIG["weight_decay"])   ### TODO -- define optimizer
    # Change scheduler from CosineAnnealingLR to OneCycleLR
    # OneCycleLR scheduler provides more tuning parameters compared to CosineAnnealingLR
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=CONFIG["learning_rate"] * 10,  # max_lr is 10x initial learning rate
        epochs=CONFIG["epochs"],
        steps_per_epoch=len(trainloader),
        pct_start=0.1,   # 10% steps for warmup
        div_factor=10,   # initial learning rate = max_lr/10
        final_div_factor=100  # final learning rate = initial lr/10
    )   ### TODO -- you can optionally add a LR scheduler

    wandb.init(project="-sp25-ds542-challenge", config=CONFIG)
    wandb.watch(model)

    ############################################################################
    # --- Training Loop (Example - Students need to complete) ---
    ############################################################################
    best_val_acc = 0.0
    for epoch in range(CONFIG["epochs"]):
        train_loss, train_acc = train(epoch, model, trainloader, optimizer, criterion, CONFIG)
        val_loss, val_acc = validate(model, valloader, criterion, CONFIG["device"])
        scheduler.step()
        
        # log to WandB
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_acc": train_acc,
            "val_loss": val_loss,
            "val_acc": val_acc,
            "lr": optimizer.param_groups[0]["lr"]
        })
        # Save the best model (based on validation accuracy)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            wandb.save("best_model.pth")
    wandb.finish()

    ############################################################################
    # Evaluation -- shouldn't have to change the following code
    ############################################################################
    import eval_cifar100
    import eval_ood
    
    # --- Evaluation on Clean CIFAR-100 Test Set ---
    predictions, clean_accuracy = eval_cifar100.evaluate_cifar100_test(model, testloader, CONFIG["device"])
    print(f"Clean CIFAR-100 Test Accuracy: {clean_accuracy:.2f}%")
    
    # --- Evaluation on OOD ---
    all_predictions = eval_ood.evaluate_ood_test(model, CONFIG)

    # --- Create Submission File (OOD) ---
    submission_df_ood = eval_ood.create_ood_df(all_predictions)
    submission_df_ood.to_csv("submission_ood_part3-50-new epochs-3.csv", index=False)
    print("submission_ood_part3-50-new epochs-3.csv created successfully.")

if __name__ == '__main__':
    main()



CONFIG Dictionary:
{'batch_size': 128,
 'data_dir': './data',
 'device': 'cuda',
 'epochs': 50,
 'learning_rate': 0.0005,
 'model': 'PretrainedResNet18',
 'num_workers': 4,
 'ood_dir': './data/ood-test',
 'seed': 42,
 'use_mixup': True,
 'wandb_project': 'sp25-ds542-challenge',
 'weight_decay': 0.0005}

Model summary:
PretrainedResNet18(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, ep

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shiychen (shiychen-boston-university). Use `wandb login --relogin` to force relogin


Epoch 1/50 [Train]:   0%|          | 0/313 [00:00<?, ?it/s]

[Validate]:   0%|          | 0/79 [00:00<?, ?it/s]

Epoch 2/50 [Train]:   0%|          | 0/313 [00:00<?, ?it/s]

...

Evaluating distortion18 (Severity 4):   0%|          | 0/79 [00:00<?, ?it/s]

Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x147721e8d510><function _MultiProcessingDataLoaderIter.__del__ at 0x147721e8d510><function _MultiProcessingDataLoaderIter.__del__ at 0x147721e8d510>


Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
  File "/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
  File "/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
            self._shutdown_workers()self._shutdown_workers()self._shutdown_workers()


  File "/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1601, in _shutdown_workers
  File "/usr4/dl4ds/shiychen/.local/lib/python3.10/site-packages/torch

distortion18 (Severity 4)


Evaluating distortion18 (Severity 5):   0%|          | 0/79 [00:00<?, ?it/s]

distortion18 (Severity 5)
submission_ood_part3-50-new epochs-3.csv created successfully.
